In [1]:
#!/usr/bin/env python3
import numpy as np

from utils import Data
from deepPC_acados import DeePC   # your Acados/CasADi DeePC implementation

# 1) Simulate a simple 2-state LTI system: x_{k+1} = A x_k + B u_k,  y_k = C x_k
A = np.array([[1.0, 1.0],
              [0.0, 1.0]])
B = np.array([[0.5],
              [1.0]])
C = np.eye(2)

T_data = 60
x = np.zeros(2)
u_data, y_data = [], []

for _ in range(T_data):
    u_k = np.random.uniform(-1, 1)
    x = A.dot(x) + (B.flatten() * u_k)
    y = C.dot(x)
    u_data.append([u_k])
    y_data.append(y)

u_data = np.vstack(u_data)   # shape (T_data × 1)
y_data = np.vstack(y_data)   # shape (T_data × 2)

data = Data(u=u_data, y=y_data)

# 2) Instantiate DeePC-Acados with Tini=10, horizon=15
Tini, horizon = 10, 15
deepc = DeePC(data, Tini=Tini, horizon=horizon, explained_variance=None)

# 3) Configure the OCP inside Acados
#    We’ll use a simple tracking cost:   Σ‖y‖²_Q  +  Σ‖u‖²_R
Qy = np.diag([10.0, 10.0])   # weight on each output channel
Ru = 1.0                     # weight on input effort
umin, umax = -1.0, 1.0       # box constraints on u

deepc.build_ocp(
    Q_y     = Qy, 
    R_u     = Ru,
    umin    = umin,
    umax    = umax,
    qp_solver               = "FULL_CONDENSING_QPOASES",
    nlp_solver_type         = "SQP",
    nlp_solver_max_iter     = 50,
    qp_solver_cond_N        = horizon
)

# 4) Prepare the “initial‐condition” data block
#    (first Tini samples become u_ini, y_ini)
data_ini = Data(
    u = u_data[:Tini, :],
    y = y_data[:Tini, :]
)

# 5) Solve the DeePC problem via Acados
u_opt, info = deepc.solve(data_ini)

print("Optimal control sequence (horizon × 1):")
print(u_opt.flatten())

print(f"\nAcados solver status: {info['solver_status']}")
print(f"Optimal objective:   {info['cost']:.4f}")


ModuleNotFoundError: No module named 'utils'

In [ ]:
import os
import time
from datetime import datetime
import threading
from pathlib import Path

import numpy as np
import scipy.io as sio
import pandas as pd

import Jetson.GPIO as GPIO
GPIO.cleanup()

import board
import busio
from adafruit_pca9685 import PCA9685

import cantools
import can
import casadi as ca
from acados_template import AcadosOcp, AcadosOcpSolver
import deepctools as dpc

from utils import *





Start loading data...
DataSheetName: 0521-sMCT1,  pwm shape: (76509,) vel shape: (76509,)
DataSheetName: 0521-sMCT2,  pwm shape: (76537,) vel shape: (76537,)
DataSheetName: 0521-sMCT3,  pwm shape: (76464,) vel shape: (76464,)
[MultiSheetTimeSeriesDataset] X shape: (229360, 50), y shape: (229360, 50)
Saved cache → save/timeseries_dataset0527.npz


NameError: name 'pyTorch_model' is not defined

In [ ]:
import numpy as np
from casadi import MX, vertcat
from acados_template import AcadosQp, AcadosQpSolver

# === 1) Offline: load data and build Hankel =========================
ud = np.load("u_data.npy")   # shape (m, T)
yd = np.load("y_data.npy")   # shape (p, T)
m, T = ud.shape
p, _ = yd.shape
T_ini = 10                   # your choice
N     = 20                   # prediction horizon
Lg    = T - (T_ini + N) + 1

def hankel(seq, rows, cols):
    # Build block-Hankel with `rows` block rows and `cols` columns
    return np.vstack([seq[:, i:i+cols] for i in range(rows)])

H_u = hankel(ud, T_ini+N, Lg)   # shape (m*(T_ini+N), Lg)
H_y = hankel(yd, T_ini+N, Lg)   # shape (p*(T_ini+N), Lg)
U_p = H_u[        :m*T_ini, :]  # shape (m*T_ini, Lg)
U_f = H_u[m*T_ini: m*(T_ini+N),:]
Y_p = H_y[        :p*T_ini, :]  # shape (p*T_ini, Lg)
Y_f = H_y[p*T_ini: p*(T_ini+N),:]

# === 2) Build QP matrices H, f =======================================
nz = Lg + m*N + p*N
# Hessian
H = np.zeros((nz, nz))
# R-blocks on u
for k in range(N):
    i = Lg + k*m
    H[i:i+m, i:i+m] = np.eye(m) * 1.0   # replace with your R
# Q-blocks on y
for k in range(N):
    j = Lg + m*N + k*p
    H[j:j+p, j:j+p] = np.eye(p) * 10.0  # replace with your Q

# gradient placeholder (we'll update for each r)
f = np.zeros(nz)

# === 3) Equality constraints A_eq z = b_eq ===========================
# rows = m*T_ini + p*T_ini + m*N + p*N
A_eq = np.zeros((m*T_ini + p*T_ini + m*N + p*N, nz))
b_eq = np.zeros((A_eq.shape[0],))

# Up * g = u_ini
A_eq[0:m*T_ini, 0: Lg]       = U_p
# Yp * g = y_ini
A_eq[m*T_ini: m*T_ini+p*T_ini, 0: Lg] = Y_p
# Uf * g - u = 0
A_eq[m*T_ini+p*T_ini: m*T_ini+p*T_ini+m*N, 0: Lg] = U_f
A_eq[m*T_ini+p*T_ini: m*T_ini+p*T_ini+m*N, Lg: Lg+m*N] = -np.eye(m*N)
# Yf * g - y = 0
i0 = m*T_ini+p*T_ini+m*N
A_eq[i0: i0+p*N, 0: Lg] = Y_f
A_eq[i0: i0+p*N, Lg+m*N: Lg+m*N+p*N] = -np.eye(p*N)

# === 4) Inequality bounds on u, y ================================
# let's say u_min <= u <= u_max, same for y
u_min, u_max = -1.0, 1.0
y_min, y_max = -5.0, 5.0

# selection matrix for u
S_u = np.zeros((m*N, nz))
for k in range(N):
    S_u[k*m:(k+1)*m, Lg + k*m: Lg+(k+1)*m] = np.eye(m)
# selection for y
S_y = np.zeros((p*N, nz))
for k in range(N):
    S_y[k*p:(k+1)*p, Lg + m*N + k*p: Lg + m*N + (k+1)*p] = np.eye(p)

# stack ineq: [ S_u; -S_u; S_y; -S_y ] * z <= [u_max; -u_min; y_max; -y_min]
A_ineq = np.vstack([S_u, -S_u, S_y, -S_y])
l_ineq = np.hstack([u_min*np.ones(m*N),
                    -u_max*np.ones(m*N),
                    y_min*np.ones(p*N),
                    -y_max*np.ones(p*N)])
u_ineq = np.hstack([u_max*np.ones(m*N),
                    -u_min*np.ones(m*N),
                    y_max*np.ones(p*N),
                    -y_min*np.ones(p*N)])

# === 5) Create and initialize acados QP solver =====================
qp = AcadosQp()
qp.solver_options.qp_solver = 'FULL_CONDENSING_HPIPM'
qp.dims.n       = nz
qp.dims.n_eq    = A_eq.shape[0]
qp.dims.n_ineq  = A_ineq.shape[0]

qp.cost.H = H
qp.cost.g = f        # will update for each r
qp.cost.idx_z = np.arange(nz)

qp.constraints.A = np.vstack([A_eq, A_ineq])
# equality rows are first
qp.constraints.lh = np.hstack([b_eq, l_ineq])
qp.constraints.uh = np.hstack([b_eq, u_ineq])

# build solver
solver = AcadosQpSolver(qp)

# === 6) At each control step: update parameters and solve ===========
def control_step(u_ini_vals, y_ini_vals, r_vals):
    # 1) update b_eq[:m*T_ini] = u_ini, next p*T_ini = y_ini
    b_eq[:m*T_ini]             = u_ini_vals.flatten()
    b_eq[m*T_ini:m*T_ini+p*T_ini] = y_ini_vals.flatten()
    solver.set('constraints.lh', np.hstack([b_eq, l_ineq]))
    solver.set('constraints.uh', np.hstack([b_eq, u_ineq]))

    # 2) update f = -2 * [zeros;    zeros;  Q*(r); ... repeated] 
    #    since J = ... + (y-r)'Q(y-r) = y'Qy - 2r'Qy + const.
    f[:] = 0
    for k in range(N):
        idx = Lg + m*N + k*p
        f[idx:idx+p] = -2 * (qp.cost.H[idx:idx+p, idx:idx+p] @ r_vals[k*p:(k+1)*p])
    solver.set('cost.g', f)

    # 3) solve
    status = solver.solve()
    z_opt = solver.get('solution')
    u_opt = z_opt[Lg: Lg+m*N]
    # apply only first m entries
    return u_opt[:m]

# === Example usage ===============================================
# suppose you have buffered the last T_ini steps into arrays
u_ini_meas = np.zeros((m, T_ini))
y_ini_meas = np.zeros((p, T_ini))
# and a reference trajectory of length N
r_traj      = np.tile(np.array([[1.0]]), (p, N))  # shape (p, N) flattened

u0 = control_step(u_ini_meas, y_ini_meas, r_traj.flatten())
print("apply u0 =", u0)


In [ ]:
import numpy as np
from acados_template import AcadosQp, AcadosQpSolver
from typing import Tuple, Optional

class DeePCAcados:
    def __init__(
        self,
        u_data: np.ndarray,
        y_data: np.ndarray,
        Tini: int,
        horizon: int,
        Q: np.ndarray,
        R: np.ndarray,
        u_bounds: Tuple[float, float],
        y_bounds: Tuple[float, float],
        lambda_g: float = 0.0,
        lambda_u: float = 0.0,
        lambda_y: float = 0.0,
    ):
        """
        :param u_data:    offline input data, shape (T, m)
        :param y_data:    offline output data, shape (T, p)
        :param Tini:      past window size
        :param horizon:   prediction horizon N
        :param Q:         (p×p) output‐error weight
        :param R:         (m×m) input‐effort weight
        :param u_bounds:  (u_min, u_max)
        :param y_bounds:  (y_min, y_max)
        :param lambda_g:  ℓ₁ penalty on g
        :param lambda_u:  ℓ₁ penalty on slack_u
        :param lambda_y:  ℓ₁ penalty on slack_y
        """
        # save dims & regularizers
        self.u_data, self.y_data = u_data, y_data
        self.T, self.m = u_data.shape
        _, self.p = y_data.shape
        self.Tini, self.N = Tini, horizon
        self.Q, self.R = Q, R
        self.u_min, self.u_max = u_bounds
        self.y_min, self.y_max = y_bounds
        self.lg, self.lu, self.ly = lambda_g, lambda_u, lambda_y

        # split data into Hankel blocks
        Lg = self.T - (Tini + horizon) + 1
        def hankel(X, rows):
            # builds block‐Hankel: rows blocks of X, each of length Lg
            return np.vstack([X[i:i+Lg].reshape(-1, Lg, order='F')
                              for i in range(rows)])
        Z_u = hankel(u_data, Tini + horizon)  # shape ((Tini+N)*m, Lg)
        Z_y = hankel(y_data, Tini + horizon)  # shape ((Tini+N)*p, Lg)

        # partition Up,Yp,Uf,Yf
        self.Up = Z_u[:m*Tini    , :]
        self.Uf = Z_u[m*Tini:     , :]
        self.Yp = Z_y[:p*Tini    , :]
        self.Yf = Z_y[p*Tini:     , :]

        # total QP dimension
        self.Lg = Lg
        self.n_u = m * horizon
        self.n_y = p * horizon
        self.n_su = m * Tini
        self.n_sy = p * Tini
        self.nz = Lg + self.n_u + self.n_y + self.n_su + self.n_sy
        self._build_acados_qp()

    def _build_acados_qp(self):
        # 1) Hessian H
        H = np.zeros((self.nz, self.nz))
        # ‖g‖₁ ≈ linear term, but here we use ℓ₂ to keep QP: approx ℓ₁→ℓ₂
        H[:self.Lg, :self.Lg] = self.lg * np.eye(self.Lg)
        # R on u‐blocks, Q on y‐blocks
        for k in range(self.N):
            iu = self.Lg + k*self.m
            jy = self.Lg + self.n_u + k*self.p
            H[iu:iu+self.m, iu:iu+self.m] = self.R
            H[jy:jy+self.p, jy:jy+self.p] = self.Q
        # slack penalties
        off_su = self.Lg + self.n_u + self.n_y
        H[off_su:off_su+self.n_su, off_su:off_su+self.n_su] = self.lu * np.eye(self.n_su)
        off_sy = off_su + self.n_su
        H[off_sy:off_sy+self.n_sy, off_sy:off_sy+self.n_sy] = self.ly * np.eye(self.n_sy)

        # 2) gradient g (updated each solve)
        g = np.zeros(self.nz)

        # 3) equality constraints A_eq z = 0  (we'll push RHS in solve)
        n_eq = self.m*self.Tini + self.p*self.Tini + self.n_u + self.n_y
        A_eq = np.zeros((n_eq, self.nz))
        # Up·g - slack_u = u_ini
        A_eq[0:self.m*self.Tini, :self.Lg] = self.Up
        A_eq[0:self.m*self.Tini, self.Lg+self.n_u+self.n_y:
                                self.Lg+self.n_u+self.n_y+self.n_su] = -np.eye(self.n_su)
        # Yp·g - slack_y = y_ini
        r1 = self.m*self.Tini
        c_sy = self.Lg + self.n_u + self.n_y + self.n_su
        A_eq[r1:r1+self.p*self.Tini, :self.Lg] = self.Yp
        A_eq[r1:r1+self.p*self.Tini, c_sy:c_sy+self.n_sy] = -np.eye(self.n_sy)
        # Uf·g - u = 0
        r2 = r1 + self.p*self.Tini
        A_eq[r2:r2+self.n_u, :self.Lg] = self.Uf
        A_eq[r2:r2+self.n_u, self.Lg:self.Lg+self.n_u] = -np.eye(self.n_u)
        # Yf·g - y = 0
        r3 = r2 + self.n_u
        A_eq[r3:r3+self.n_y, :self.Lg] = self.Yf
        A_eq[r3:r3+self.n_y, self.Lg+self.n_u:self.Lg+self.n_u+self.n_y] = -np.eye(self.n_y)

        # 4) box bounds on u,y
        zl = -np.inf*np.ones(self.nz)
        zu =  np.inf*np.ones(self.nz)
        # u‐bounds
        for k in range(self.N):
            iu = self.Lg + k*self.m
            zl[iu:iu+self.m] = self.u_min
            zu[iu:iu+self.m] = self.u_max
        # y‐bounds
        for k in range(self.N):
            jy = self.Lg + self.n_u + k*self.p
            zl[jy:jy+self.p] = self.y_min
            zu[jy:jy+self.p] = self.y_max

        # 5) create acados QP
        qp = AcadosQp()
        qp.solver_options.qp_solver = 'FULL_CONDENSING_HPIPM'
        qp.dims.n      = self.nz
        qp.dims.n_eq   = n_eq
        qp.dims.n_ineq = 0  # no extra linear inequalities
        qp.cost.H      = H
        qp.cost.g      = g
        qp.cost.idx_z  = np.arange(self.nz)
        qp.constraints.A  = A_eq
        qp.constraints.lh = np.zeros(n_eq)
        qp.constraints.uh = np.zeros(n_eq)
        qp.constraints.zl = zl
        qp.constraints.zu = zu

        # build solver once
        self._solver = AcadosQpSolver(qp)

    def solve(self,
              u_ini: np.ndarray,
              y_ini: np.ndarray,
              r: np.ndarray
    ) -> np.ndarray:
        """
        :param u_ini:  shape (Tini, m)
        :param y_ini:  shape (Tini, p)
        :param r:      reference trajectory, shape (N, p)
        :return u0:    first control move, shape (m,)
        """
        # 1) update RHS of eqs: A_eq z = [u_ini; y_ini; 0; 0]
        b_eq = np.hstack([
            u_ini.flatten(),
            y_ini.flatten(),
            np.zeros(self.n_u),
            np.zeros(self.n_y),
        ])
        self._solver.set('constraints.lh', b_eq)
        self._solver.set('constraints.uh', b_eq)

        # 2) update gradient for reference r: grad_y = -2 Q r_k
        g = np.zeros(self.nz)
        for k in range(self.N):
            idx = self.Lg + self.n_u + k*self.p
            g[idx:idx+self.p] = -2 * (self.Q @ r[k])
        self._solver.set('cost.g', g)

        # 3) solve and extract
        status = self._solver.solve()
        z_opt = self._solver.get('solution')
        u_opt = z_opt[self.Lg : self.Lg + self.m*self.N]
        return u_opt.reshape(self.N, self.m)[0]   # return only u(0)



In [ ]:
# ─── LOAD DATA FOR DeePC ───────────────────────────────────────────────
xlsx_path="PWMDynoSpeedNewsMCT.xlsx"
seq_len = 50
ds = MultiSheetTimeSeriesDataset(xlsx_path, seq_len, normalize=True, cache_path="save/timeseries_dataset0527.npz")

u0 = deepc.solve(u_ini=u_past, y_ini=y_past, r=ref_traj)


In [ ]:
import numpy as np
from typing import NamedTuple, Tuple, Optional, List, Union
from cvxpy import Expression, Variable, Problem, Parameter
from cvxpy.constraints.constraint import Constraint
from numpy.typing import NDArray

class OptimizationProblemVariables(NamedTuple):
    """
    Class used to store all the variables used in the optimization
    problem
    """
    u_ini: Union[Variable, Parameter]
    y_ini: Union[Variable, Parameter]
    u: Union[Variable, Parameter]
    y: Union[Variable, Parameter]
    g: Union[Variable, Parameter]
    slack_y: Union[Variable, Parameter]
    slack_u: Union[Variable, Parameter]


class OptimizationProblem(NamedTuple):
    """
    Class used to store the elements an optimization problem
    :param problem_variables:   variables of the opt. problem
    :param constraints:         constraints of the problem
    :param objective_function:  objective function
    :param problem:             optimization problem object
    """
    variables: OptimizationProblemVariables
    constraints: List[Constraint]
    objective_function: Expression
    problem: Problem

class Data(NamedTuple):
    """
    Tuple that contains input/output data
    :param u: input data
    :param y: output data
    """
    u: NDArray[np.float64]
    y: NDArray[np.float64]


def create_hankel_matrix(data: NDArray[np.float64], order: int) -> NDArray[np.float64]:
    """
    Create an Hankel matrix of order L from a given matrix of size TxM,
    where M is the number of features and T is the batch size.
    Note that we need L <= T.

    :param data:  A matrix of data (size TxM). 
                  T is the batch size and M is the number of features
    :param order: The order of the Hankel matrix (L)
    :return:      The Hankel matrix of type np.ndarray
    """
    data = np.array(data)
    
    assert len(data.shape) == 2, "Data needs to be a matrix"

    T, M = data.shape
    assert T >= order and order > 0, "The number of data points needs to be larger than the order"

    H = np.zeros((order * M, (T - order + 1)))
    for idx in range (T - order + 1):
        H[:, idx] = data[idx:idx+order, :].flatten()
    return H

def split_data(data: Data, Tini: int, horizon: int, explained_variance: Optional[float] = None) -> Tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
    """
    Utility function used to split the data into past data and future data.
    Constructs the Hankel matrix for the input/output data, and uses the first
    Tini rows to create the past data, and the last 'horizon' rows to create
    the future data.
    For more info check eq. (4) in https://arxiv.org/pdf/1811.05890.pdf

    :param data:                A tuple of input/output data. Data should have shape TxM
                                where T is the batch size and M is the number of features
    :param Tini:                number of samples needed to estimate initial conditions
    :param horizon:             horizon
    :param explained_variance:  Regularization term in (0,1] used to approximate the Hankel matrices.
                                By default is None (no low-rank approximation is performed).
    :return:                    Returns Up,Uf,Yp,Yf (see eq. (4) of the original DeePC paper)
    """
    assert Tini >= 1, "Tini cannot be lower than 1"
    assert horizon >= 1, "Horizon cannot be lower than 1"
    assert explained_variance is None or 0 < explained_variance <= 1, "explained_variance should be in (0,1] or be none"
 
    Mu, My = data.u.shape[1], data.y.shape[1]
    Hu = create_hankel_matrix(data.u, Tini + horizon)
    Hy = create_hankel_matrix(data.y, Tini + horizon)

    if explained_variance is not None:
        Hu = low_rank_matrix_approximation(Hu, explained_var=explained_variance)
        Hy = low_rank_matrix_approximation(Hy, explained_var=explained_variance)

    Up, Uf = Hu[:Tini * Mu], Hu[-horizon * Mu:]
    Yp, Yf = Hy[:Tini * My], Hy[-horizon * My:]
    
    return Up, Uf, Yp, Yf

def low_rank_matrix_approximation(
        X:NDArray[np.float64],
        explained_var: Optional[float] = 0.9,
        rank: Optional[int] = None,
        SVD: Optional[Tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]] = None,
        **svd_kwargs) -> NDArray[np.float64]:
    """
    Computes an low-rank approximation of a matrix

    Adapted from https://gist.github.com/thearn/5424219

    :param X:               matrix to approximate
    :param explained_var:   Value in (0,1] used to compute the rank. Describes how close 
                            the low rank matrix is to the original matrix (in terms of the
                            singular values). Default value: 0.9
    :param rank:            rank order. To be used if you want to approximate the matrix by a specific
                            rank. By default is None. If different than None, then it will override the
                            explained_var parameter.
    :param SVD:             If not none, it should be the SVD decomposition (see numpy.linalg.svd) of X
    :param **svd_kwargs:    additional parameters to be passed to numpy.linalg.svd
    :return: the low rank approximation of X
    """
    assert len(X.shape) == 2, "X must be a matrix"
    assert isinstance(rank, tuple([int, float])) or isinstance(explained_var, tuple([int, float])), \
        "You need to specify explained_var or rank!"
    assert explained_var is None or explained_var <= 1. and explained_var > 0, \
        "explained_var must be in (0,1]"
    assert rank is None or (rank >= 1 and rank <= min(X.shape[0], X.shape[1])), \
        "Rank cannot be lower than 1 or greater than min(num_rows, num_cols)"
    assert SVD is None or len(SVD) == 3, "SVD must be a tuple of 3 elements"

    u, s, v = np.linalg.svd(X, full_matrices=False, **svd_kwargs) if not SVD else SVD

    if rank is None:
        s_squared = np.power(s, 2)
        total_var = np.sum(s_squared)
        z = np.cumsum(s_squared) / total_var
        rank = np.argmax(np.logical_or(z > explained_var, np.isclose(z, explained_var)))

    u_low = u[:,:rank]
    s_low = s[:rank]
    v_low = v[:rank,:]

    X_low = np.dot(u_low * s_low, v_low)
    return X_low
